In [2]:
import bnlearn as bn
import pandas as pd
import numpy as np

import logging

# Reduce the console logging level for these libraries
logging.getLogger("bnlearn").setLevel(logging.ERROR)

###########################
# 1. Coverage Fraction
###########################
def coverage_fraction(candidate_parents, sampled_dags):
    """
    Returns the fraction of sampled DAGs whose parent sets
    are all subsets of candidate_parents[node].
    """
    count_covered = 0
    total = len(sampled_dags)
    
    for dag in sampled_dags:
        covered = True
        for node, parents in dag.items():
            # If the DAG's parents for this node go beyond what
            # the candidate set allows, it's not covered
            if not parents.issubset(candidate_parents[node]):
                covered = False
                break
        if covered:
            count_covered += 1
    
    return count_covered / total if total > 0 else 0.0

###########################
# 2. Repeated Pipeline
###########################
def estimate_expected_coverage(
    n_reps,            # how many datasets to sample
    n_samples_data,    # how many data points in each dataset
    model_path,        # path to a .bif file or other BN structure
    coverage_fn,       # function, e.g. coverage_fraction
    heuristic_fn=None, # function to compute candidate_parents from data, if needed
    dag_learning_fn=None,  # function to learn/sample DAGs from data
):
    """
    Repeatedly:
      1) Generate or sample data from a known BN model (or from some process).
      2) Use 'heuristic_fn' to get candidate parents (optional).
      3) Learn or sample DAGs (using 'dag_learning_fn').
      4) Compute coverage fraction.
    Return the average coverage fraction across multiple data draws.
    """

    # 1) Load or import a Bayesian network structure (for simulating data)
    hailfinder_model = bn.import_DAG(model_path)

    coverage_values = []
    for rep in range(n_reps):
        print(f"=== Data draw #{rep+1} of {n_reps} ===")

        # 2) Simulate a fresh dataset from the BN
        df_data = bn.sampling(hailfinder_model, n=n_samples_data)
        df_data = pd.DataFrame(df_data)

        # 3) If your candidate parent heuristic depends on data, compute it here:
        if heuristic_fn is not None:
            candidate_parents = heuristic_fn(df_data)
        else:
            # If you already have a fixed candidate_parents, define it outside
            # or just do something trivial for demonstration:
            n_nodes = df_data.shape[1]
            candidate_parents = {i: set(range(n_nodes)) - {i} for i in range(n_nodes)}

        # 4) Learn or sample DAGs from this data
        if dag_learning_fn is not None:
            sampled_dags = dag_learning_fn(df_data)
        else:
            # Example fallback: produce a single empty DAG (no edges).
            n_nodes = df_data.shape[1]
            dag = {i: set() for i in range(n_nodes)}
            sampled_dags = [dag]

        # 5) Compute coverage fraction for this data’s DAGs
        fraction = coverage_fn(candidate_parents, sampled_dags)
        coverage_values.append(fraction)

    # 6) Average coverage fraction across all data draws
    return np.mean(coverage_values)

###########################
# 3. Demo Heuristic / DAG Learning
###########################
def dummy_heuristic(df_data):
    """
    Example heuristic that returns "every other node is a candidate parent."
    Replace this with your real heuristic, e.g., sumu's 'pc', 'mb', etc.
    """
    n_nodes = df_data.shape[1]
    candidate_parents = {}
    for node in range(n_nodes):
        # Just a silly example: candidate parents are all nodes with odd index if node is even, etc.
        # Replace with something meaningful in your real code.
        if node % 2 == 0:
            candidate_parents[node] = {i for i in range(n_nodes) if i % 2 == 1 and i != node}
        else:
            candidate_parents[node] = {i for i in range(n_nodes) if i % 2 == 0 and i != node}
    return candidate_parents

def dummy_dag_learning(df_data):
    """
    Example DAG-learning function that returns:
       1) an empty DAG, plus
       2) a random DAG with one parent assigned arbitrarily.

    Replace with Gobnilp, R MCMC, GES, or any real pipeline.
    """
    n_nodes = df_data.shape[1]
    
    # DAG #1: empty
    dag_empty = {i: set() for i in range(n_nodes)}

    # DAG #2: pick a random parent for each node (except for node=0)
    dag_random = {}
    for node in range(n_nodes):
        # trivial logic: let's pick a parent by random from [0..n_nodes-1], ignoring self
        possible_parents = list(range(n_nodes))
        possible_parents.remove(node)
        # choose one random parent or none:
        chosen_parent = np.random.choice(possible_parents)
        # 50% chance to have no parent, 50% chance to have chosen_parent
        if np.random.rand() < 0.5:
            dag_random[node] = set()
        else:
            dag_random[node] = {chosen_parent}

    return [dag_empty, dag_random]

###########################
# 4. Run the Test
###########################
if __name__ == "__main__":
    # Path to Hailfinder model or any .bif you want to use for testing
    model_path = "data/hailfinder.bif" 

    # We do 3 repeated draws, each time sampling 500 data points
    n_reps = 3
    n_samples_data = 500

    # Call the repeated pipeline
    avg_coverage = estimate_expected_coverage(
        n_reps=n_reps,
        n_samples_data=n_samples_data,
        model_path=model_path,
        coverage_fn=coverage_fraction,    # our coverage metric
        heuristic_fn=dummy_heuristic,     # or None if you prefer
        dag_learning_fn=dummy_dag_learning
    )

    print("\n=== Result ===")
    print("Estimated coverage across multiple data draws:", avg_coverage)


[bnlearn] >Import <data/hailfinder.bif>
[bnlearn] >Loading bif file <data/hailfinder.bif>


[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.


[bnlearn] >Check whether CPDs sum up to one.
[bnlearn] >CPD [AMCINInScen] does not add up to 1 but is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[bnlearn] >CPD [AMInsWliScen] does not add up to 1 but is: [[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
[bnlearn] >CPD [CapInScen] does not add up to 1 but is: [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[bnlearn] >CPD [CldShadeConv] does not add up to 1 but is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[bnlearn] >CPD [CombVerMo] does not add up to 1 but is: [[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]

[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: 2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: 1.1102230246251565e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: 2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values

=== Data draw #2 of 3 ===


[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability valu

=== Data draw #3 of 3 ===


[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability values don't exactly sum to 1. Differ by: -2.220446049250313e-16. Adjusting values.
[setgraphviz] >WARNING> Probability valu


=== Result ===
Estimated coverage across multiple data draws: 0.5
